<a href="https://colab.research.google.com/github/ahmeeeeedddddd/Predictify/blob/main/Teeth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle
!pip install -q monai opencv-python matplotlib diffusers accelerate
import numpy as np
from PIL import Image
from google.colab import files
import os
import cv2
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import layers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.6 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
tf.keras.backend.clear_session()

# Confirm that GPU is visible to TensorFlow
print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU devices: []


In [10]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1) (1).json


{'kaggle (1) (1).json': b'{"username":"ahmeeeedddd","key":"084444bcaf4bc3c7f83088ba469c8161"}'}

In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d humansintheloop/teeth-segmentation-on-dental-x-ray-images

Dataset URL: https://www.kaggle.com/datasets/humansintheloop/teeth-segmentation-on-dental-x-ray-images
License(s): CC0-1.0
 98% 4.07G/4.14G [01:28<00:04, 14.9MB/s]
100% 4.14G/4.14G [01:29<00:00, 50.0MB/s]


In [14]:
!unzip teeth-segmentation-on-dental-x-ray-images.zip -d teeth_segmentation_data

Archive:  teeth-segmentation-on-dental-x-ray-images.zip
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/1.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/10.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/100.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/101.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/102.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/103.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/104.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/105.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/106.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/107.jpg.json  
  inflating: teeth_segmentation_data/Teeth Segmentation JSON/d2/ann/108.jpg.json  
  inflating: teeth_segmentation_da

In [ ]:
for root, dirs, files in os.walk("/kaggle"):
    print(root)

NameError: name 'os' is not defined

In [ ]:
!ls -R teeth_segmentation_data

teeth_segmentation_data:
'Teeth Segmentation JSON'  'Teeth Segmentation PNG'

'teeth_segmentation_data/Teeth Segmentation JSON':
d2  meta.json  obj_class_to_machine_color.json

'teeth_segmentation_data/Teeth Segmentation JSON/d2':
ann  img  masks_human  masks_machine

'teeth_segmentation_data/Teeth Segmentation JSON/d2/ann':
100.jpg.json  209.jpg.json  317.jpg.json  425.jpg.json	533.jpg.json
101.jpg.json  20.jpg.json   318.jpg.json  426.jpg.json	534.jpg.json
102.jpg.json  210.jpg.json  319.jpg.json  427.jpg.json	535.jpg.json
103.jpg.json  211.jpg.json  31.jpg.json   428.jpg.json	536.jpg.json
104.jpg.json  212.jpg.json  320.jpg.json  429.jpg.json	537.jpg.json
105.jpg.json  213.jpg.json  321.jpg.json  42.jpg.json	538.jpg.json
106.jpg.json  214.jpg.json  322.jpg.json  430.jpg.json	539.jpg.json
107.jpg.json  215.jpg.json  323.jpg.json  431.jpg.json	53.jpg.json
108.jpg.json  216.jpg.json  324.jpg.json  432.jpg.json	540.jpg.json
109.jpg.json  217.jpg.json  325.jpg.json  433.jpg.json	541.jpg.

Training segmentation model

In [15]:
IMG_SIZE = 256  # Resize all images to 256x256
IMG_DIR = "teeth_segmentation_data/Teeth Segmentation JSON/d2/img"
MASK_DIR = "teeth_segmentation_data/Teeth Segmentation JSON/d2/masks_human"
print("Number of images:", len(os.listdir(IMG_DIR)))
print("Number of masks:", len(os.listdir(MASK_DIR)))

Number of images: 598
Number of masks: 598


In [16]:
def load_image_mask_pairs(img_dir, mask_dir, size=256, limit=None):
    images, masks = [], []
    filenames = sorted(os.listdir(img_dir))[:limit]

    for fname in filenames:
        img_path = os.path.join(img_dir, fname)
        mask_path = os.path.join(mask_dir, fname.replace(".jpg", ".png"))

        if os.path.exists(img_path) and os.path.exists(mask_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (size, size))
            img = img / 255.0

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (size, size))
            mask = mask / 255.0
            mask = np.expand_dims(mask, axis=-1)

            images.append(img)
            masks.append(mask)

    return np.array(images), np.array(masks)


In [17]:
X, Y = load_image_mask_pairs(IMG_DIR, MASK_DIR, size=IMG_SIZE)  # No limit
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [18]:
def build_unet(input_shape):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2,2))(c1)

    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2,2))(c2)

    # Bottleneck
    b = layers.Conv2D(256, (3,3), activation='relu', padding='same')(p2)
    b = layers.Conv2D(256, (3,3), activation='relu', padding='same')(b)

    # Decoder
    u1 = layers.UpSampling2D((2,2))(b)
    u1 = layers.Concatenate()([u1, c2])
    c3 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c3)

    u2 = layers.UpSampling2D((2,2))(c3)
    u2 = layers.Concatenate()([u2, c1])
    c4 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c4)

    outputs = layers.Conv2D(1, (1,1), activation='sigmoid')(c4)

    return tf.keras.Model(inputs, outputs)


In [19]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


In [20]:
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

In [21]:
model = build_unet((IMG_SIZE, IMG_SIZE, 3))
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient, iou])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 128, 128,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 384)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 128, 128,  │    442,496 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 256, 256,  │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 192)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 256, 256,  │    110,656 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d_8[0][0]  

 Total params: 1,883,137 (7.18 MB)

 Trainable params: 1,883,137 (7.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=10,
    batch_size=16
)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1977s 66s/step - dice_coefficient: 0.4933 - iou: 0.3282 - loss: 0.5067 - val_dice_coefficient: 0.5156 - val_iou: 0.3475 - val_loss: 0.4849
Epoch 2/10
 8/30 ━━━━━━━━━━━━━━━━━━━━ 22:08 60s/step - dice_coefficient: 0.5110 - iou: 0.3432 - loss: 0.4890

In [ ]:

def show_sample(image, mask, prediction, threshold=True):
    """
    Displays the input image, ground truth mask, and model prediction.

    Parameters:
    - image: RGB input image
    - mask: Ground truth binary mask
    - prediction: Predicted mask from model (with sigmoid output)
    - threshold: Whether to apply 0.5 threshold to prediction
    """
    # Ensure prediction has correct shape
    if prediction.shape[-1] == 1:
        prediction = prediction[:, :, 0]

    if threshold:
        prediction = (prediction > 0.5).astype(np.float32)

    plt.figure(figsize=(15, 5))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(image.astype(np.uint8))
    plt.title("Input Image")
    plt.axis('off')

    # Ground truth mask
    plt.subplot(1, 3, 2)
    plt.imshow(mask[:, :, 0], cmap='gray')
    plt.title("Ground Truth Mask")
    plt.axis('off')

    # Predicted mask
    plt.subplot(1, 3, 3)
    plt.imshow(prediction, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Run prediction on validation set
preds = model.predict(X_val)

# Show a sample (index 0 in this case)
show_sample(X_val[0], Y_val[0], preds[0])


In [ ]:
model.save("teeth_segmentation_unet.h5")


Main code-segmentation is now done with the trained model

In [ ]:
# ========== SECTION SEPARATORS ========== #
def print_section_header(title):
    print("\n" + "="*80)
    print(f" {title.upper()} ".center(80, '='))
    print("="*80 + "\n")

# ========== MAIN WORKFLOW ========== #
print_section_header("Setup Environment")

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ========== UPLOAD IMAGE ========== #
print_section_header("Image Upload")

print("Please upload your dental X-ray/CT image (PNG or JPG format)")
uploaded = files.upload()

# Get the uploaded filename
img_path = list(uploaded.keys())[0] if uploaded else None

if not img_path:
    # Use sample image if no upload
    !wget https://raw.githubusercontent.com/Project-MONAI/MONAI-extra-test-data/main/panoramic_dental.png
    img_path = "panoramic_dental.png"
    print("Using sample image: panoramic_dental.png")
else:
    print(f"Using uploaded image: {img_path}")

# Read the image
img = cv2.imread(img_path, 0)  # Read as grayscale
if img is None:
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Failed to load image. Please try another format.")

print(f"\nImage loaded successfully. Dimensions: {img.shape}")
plt.figure(figsize=(8,6))
plt.imshow(img, cmap='gray')
plt.title("Your Dental Image")
plt.show()

# ========== SEGMENTATION ========== #
print_section_header("Teeth Segmentation")

img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise ValueError("Failed to load image. Please try another image.")

print("Preprocessing image...")
img_preprocessed = cv2.equalizeHist(img)
img_preprocessed = cv2.GaussianBlur(img_preprocessed, (7,7), 0)

# ========== 🔄 REPLACED THIS BLOCK WITH MODEL PREDICTION ========== #
print("Predicting mask using trained U-Net...")

# Resize to model input size
img_resized = cv2.resize(img_preprocessed, (IMG_SIZE, IMG_SIZE))
img_input = img_resized.astype(np.float32) / 255.0
img_input = np.expand_dims(img_input, axis=-1)  # Add channel dimension
img_input = np.repeat(img_input, 3, axis=-1)     # If model was trained on 3-channel
img_input = np.expand_dims(img_input, axis=0)    # Add batch dimension

# Predict mask
pred_mask = model.predict(img_input)[0]
pred_mask = (pred_mask > 0.5).astype(np.uint8)

# Resize prediction to original image size
mask = cv2.resize(pred_mask[:, :, 0], (img.shape[1], img.shape[0]))

# Post-process mask
print("Cleaning up prediction mask...")
kernel = np.ones((5,5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

print("\nSegmentation Results:")
plt.figure(figsize=(15,10))
plt.subplot(131), plt.imshow(img, cmap='gray'), plt.title('Original')
plt.subplot(132), plt.imshow(pred_mask[:, :, 0], cmap='gray'), plt.title('Raw Prediction')
plt.subplot(133), plt.imshow(mask, cmap='jet'), plt.title('Final Mask')
plt.tight_layout()
plt.show()

unique, counts = np.unique(mask, return_counts=True)
print("\nMask Statistics:")
print(f"Total pixels: {mask.size}")
print(f"Non-zero pixels: {np.count_nonzero(mask)}")
print(f"Mask coverage: {np.count_nonzero(mask)/mask.size*100:.2f}%")

# ========== 3D RECONSTRUCTION ========== #
print_section_header("3D Reconstruction")

print("Creating depth map...")
depth_map = np.zeros_like(img, dtype=np.float32)
center_x = img.shape[1] // 2
center_y = img.shape[0] // 2

for y in range(img.shape[0]):
    for x in range(img.shape[1]):
        if mask[y, x] > 0:
            horizontal_factor = ((x - center_x) / center_x)**2
            vertical_factor = 1 - (y / img.shape[0])
            depth_value = horizontal_factor + vertical_factor
            depth_map[y, x] = depth_value

plt.figure(figsize=(10,6))
plt.imshow(depth_map, cmap='viridis')
plt.colorbar(label='Depth Value')
plt.title("Depth Map Visualization")
plt.show()

print("Creating point cloud...")
points = []
for y in range(img.shape[0]):
    for x in range(img.shape[1]):
        if mask[y, x] > 0:
            points.append([x, y, depth_map[y, x] * 10])

if len(points) == 0:
    print("Creating fallback dental arch")
    height, width = img.shape
    for x in range(100, width-100, 5):
        arch_curve = 0.005 * (x - width/2)**2
        for y in range(100, height-100, 5):
            if abs(y - (height/2 + arch_curve * 50)) < 30:
                points.append([x, y, 0])

point_cloud = np.array(points)
print(f"Created point cloud with {len(points)} points")

if len(points) > 0:
    print("\n3D Point Cloud Visualization:")
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    z_values = point_cloud[:, 2]
    sc = ax.scatter(point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2],
                    c=z_values, cmap='viridis', s=5, alpha=0.7)
    ax.set_xlabel('X (Left-Right)')
    ax.set_ylabel('Y (Top-Bottom)')
    ax.set_zlabel('Depth (Front-Back)')
    ax.set_title('3D Dental Reconstruction')
    cbar = fig.colorbar(sc, ax=ax, pad=0.1)
    cbar.set_label('Depth Value', rotation=270, labelpad=15)
    ax.view_init(elev=25, azim=-60)
    ax.set_box_aspect([1, 1, 0.5])
    plt.tight_layout()
    plt.savefig('3d_reconstruction.png', dpi=300)
    plt.show()
else:
    print("Error: Still no points for visualization")

# ========== MOVEMENT PREDICTION ========== #
print_section_header("Movement Prediction")

# ✅ GPU-accelerated movement simulation
def simulate_tooth_movement(initial_points, years=1, pressure_map=None):
    current_points = torch.tensor(initial_points, dtype=torch.float32, device=device)
    months = years * 12

    for month in range(months):
        movement_rate = 0.04
        displacement = movement_rate * torch.randn_like(current_points)

        if pressure_map is not None:
            pressure = torch.tensor(pressure_map, dtype=torch.float32, device=device)
            displacement[:, 0] *= pressure[:, 0]
            displacement[:, 1] *= pressure[:, 1]

        current_points += displacement

        if years > 2:
            growth_factor = 0.01 * np.sin(month / 12)
            current_points[:, 1] += growth_factor

    return current_points.cpu().numpy()

print("Creating pressure map...")
pressure_map = np.zeros_like(point_cloud)
pressure_map[:, 0] = np.random.uniform(-0.5, 0.5, size=point_cloud.shape[0])
pressure_map[:, 1] = np.random.uniform(-0.2, 0.2, size=point_cloud.shape[0])

print("Simulating tooth movement for 5 years...")
yearly_point_clouds = []
for year in range(1, 6):
    print(f"\nSimulating Year {year}...")
    year_points = simulate_tooth_movement(point_cloud, year, pressure_map)
    yearly_point_clouds.append(year_points)

    plt.figure(figsize=(10, 7))
    ax = plt.subplot(111, projection='3d')
    ax.scatter(point_cloud[:,0], point_cloud[:,1], point_cloud[:,2],
               s=5, c='b', label='Original')
    ax.scatter(year_points[:,0], year_points[:,1], year_points[:,2],
               s=5, c='r', label=f'Year {year}')
    ax.legend()
    ax.set_title(f'Tooth Movement Prediction - Year {year}')
    plt.tight_layout()
    plt.savefig(f'movement_year_{year}.png')
    plt.show()

print("\nMovement simulation complete!")
